In [2]:
# Importing necessary libraries
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase Admin SDK with your service account key
cred = credentials.Certificate('fmsdb-e4283-firebase-adminsdk-7jzkd-f40c56c494.json')
firebase_admin.initialize_app(cred)

# Initialize Firestore
db = firestore.client()


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore  # Use Firestore database

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

# URL of the webpage containing the table
url = "https://infobencanajkmv2.jkm.gov.my/landing/index.php?a=1#!"
driver.get(url)

# Locate the table using its id
table = driver.find_element(By.ID, "overviewPPS")

# Locate all rows within the table
rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # [1:] to skip the header row

# Initialize an empty list to store the data
data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = []
    for cell in cells:
        row_data.append(cell.text)  # Extracting text from each cell
    data.append(row_data)

# Convert the data into a Pandas DataFrame
df = pd.DataFrame(data, columns=["Cari",
    "Nama","Cari", "Daerah", "Mukim", "Buka", "Kapasiti", "Mangsa", 
    "Keluarga", "Lelaki Dewasa", "Perempuan Dewasa", 
    "Kanak-kanak Lelaki", "Kanak-kanak Perempuan", "Bayi Lelaki", "Bayi Perempuan",
])

# Remove the "Cari" column if it exists
if "Cari" in df.columns:
    df.drop("Cari", axis=1, inplace=True)
    
df.rename(columns={
    "Nama": "name",
    "Daerah": "district",
    "Mukim": "subdistrict",
    "Buka": "open",
    "Kapasiti": "capacity",
    "Mangsa": "victims",
    "Keluarga": "families",
    "Lelaki Dewasa": "men",
    "Perempuan Dewasa": "women",
    "Kanak-kanak Lelaki": "boys",
    "Kanak-kanak Perempuan": "girls",
    "Bayi Lelaki": "baby_boys",
    "Bayi Perempuan": "baby_girls",
}, inplace=True)

# Save to CSV (optional)
df.to_csv("table_data.csv", index=False)

pps_collection = db.collection('pps')

# Add each row as a document in the 'pps' collection
for _, row in df.iterrows():
    pps_collection.document(row['name']).set(row.to_dict())

print("Data has been stored in Firebase Firestore")

# Close the browser
driver.quit()


Data has been stored in Firebase Firestore
